In [8]:
import pandas as pd
import numpy as np
from sklearn import neighbors

In [9]:
# Init
n_neighbors = 5


In [20]:
train_data = pd.read_csv('mnist_train.csv',header=None)
test_data = pd.read_csv('mnist_test.csv',header=None)

In [27]:
train_y = train_data.iloc[:,0].values
test_y = test_data.iloc[:,0].values

In [28]:
test_y

array([5, 0, 4, ..., 5, 6, 8])

In [10]:
# Let's try KNN
clf = neighbors.KNeighborsClassifier(n_neighbors)

In [ ]:
# Fit the model
clf.fit(X, y)